# [`wordplay` 🎮 💬](https://github.com/saforem2/wordplay): Shakespeare

[Sam Foreman](https://samforeman.me)
(\[[ALCF](https://alcf.anl.gov/about/people/sam-foreman)\](<https://alcf.anl.gov/about/people/sam-foreman>))  
2025-07-22

✍️

[Sam Foreman](https://samforeman.me)
(\[[ALCF](https://alcf.anl.gov/about/people/sam-foreman)\](<https://alcf.anl.gov/about/people/sam-foreman>))  
2025-07-22

We will be using the [Shakespeare
dataset](https://github.com/saforem2/wordplay/blob/main/data/shakespeare/readme.md)
to train a (~ small) 10M param LLM *from scratch*.

<img src="https://github.com/saforem2/wordplay/blob/main/assets/shakespeare.jpeg?raw=true" width="45%" align="center" /><br>

Image generated from
[stabilityai/stable-diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
on [🤗 Spaces](https://huggingface.co/spaces).<br>

Prompt Details

Prompt:

<t><q> Shakespeare himself, dressed in full Shakespearean garb, writing
code at a modern workstation with multiple monitors, hacking away
profusely, backlit, high quality for publication </q></t>

Negative Prompt:

<t><q> low quality, 3d, photorealistic, ugly </q></t>

## Install / Setup

<b>Warning!</b><br>

**IF YOU ARE EXECUTING ON GOOGLE COLAB**:

You will need to restart your runtime (`Runtime` $\rightarrow\,$
`Restart runtime`)  
*after* executing the following cell:

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/content/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.

## Post Install

If installed correctly, you should be able to:

``` python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import ezpz

os.environ['COLORTERM'] = 'truecolor'
if sys.platform == 'darwin':
    # If running on MacOS:
    # os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    os.environ['TORCH_DEVICE'] = 'cpu'
# -----------------------------------------------

logger = ezpz.get_logger()

import wordplay
logger.info(wordplay.__file__)

[ 2025-07-23 17:07:07,066155 ][ I ][ ezpz / __init__ : 265 : ezpz ] Setting logging level to 'INFO' on 'RANK == 0'

[ 2025-07-23 17:07:07,072771 ][ I ][ ezpz / __init__ : 266 : ezpz ] Setting logging level to 'CRITICAL' on all others 'RANK != 0'

[ 2025-07-23 17:07:07,079375 ][ I ][ tmp / ipython-input-2-2338663768 : 17 : ezpz.log ] /content/wordplay/src/wordplay/ __init__.py

## Build Trainer

Explicitly, we:

1.  `setup_torch(...)`
2.  Build `cfg: DictConfig = get_config(...)`
3.  Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4.  Build `trainer = Trainer(config)`

In [3]:
import wordplay
print(wordplay.__file__)

/content/wordplay/src/wordplay/__init__.py

In [4]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'model.batch_size=8',
        'model.block_size=1024',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=bfloat16',
        'train.max_iters=1000',
        'train.log_interval=10',
        'train.eval_interval=100',
    ]
)
config = instantiate(cfg)

[ 2025-07-23 17:07:07,409437 ][ I ][ wordplay / configs : 81 ] Setting HF_DATASETS_CACHE to /content/wordplay/.cache/huggingface/ datasets

[ 2025-07-23 17:07:07,435593 ][ I ][ ezpz / dist : 1159 ] Using fw = 'ddp' with torch_ { device,backend } = { cuda, nccl }

[ 2025-07-23 17:07:07,438150 ][ I ][ ezpz / dist : 1026 ] Caught MASTER_PORT = 41765 from environment!

[ 2025-07-23 17:07:07,440989 ][ I ][ ezpz / dist : 1042 ] Using torch.distributed.init_process_group with
- master_addr = '588b3fb1cb70' 
- master_port = '41765' 
- world_size = 1 
- rank = 0 
- local_rank = 0 
- timeout = datetime.timedelta ( seconds = 3600 ) 
- backend = 'nccl'

[ 2025-07-23 17:07:07,447590 ][ I ][ ezpz / dist : 759 ] Calling torch.distributed.init_process_group_with: rank = 0 world_size = 1 backend = nccl

...

[ 2025-07-23 17:07:08,215788 ][ I ][ wordplay / configs : 317 ] Loading val from /content/wordplay/data/shakespeare_char/ val.bin

[ 2025-07-23 17:07:08,221368 ][ I ][ wordplay / configs : 317 ] Loading train from /content/wordplay/data/shakespeare_char/ train.bin

[ 2025-07-23 17:07:08,226696 ][ I ][ wordplay / configs : 442 ] Tokens per iteration: 8 , 192

[ 2025-07-23 17:07:08,231221 ][ I ][ wordplay / configs : 465 ] Using self. ptdtype = torch .bfloat16 on self. device_type = 'cuda'

[ 2025-07-23 17:07:08,234866 ][ I ][ wordplay / configs : 471 ] Initializing a new model from scratch

### Build `Trainer` object

In [5]:
trainer = Trainer(config)

[ 2025-07-23 17:07:08,315621 ][ I ][ wordplay / trainer : 248 ] Initializing a new model from scratch

[ 2025-07-23 17:07:08,654618 ][ I ][ wordplay / model : 255 ] number of parameters: 10. 65M

[ 2025-07-23 17:07:08,675995 ][ I ][ wordplay / trainer : 266 ] Model size: num_params = 10646784

[ 2025-07-23 17:07:08,686453 ][ I ][ wordplay / model : 445 ] num decayed parameter tensors: 26 , with 11 , 035 , 008 parameters

[ 2025-07-23 17:07:08,690282 ][ I ][ wordplay / model : 449 ] num non-decayed parameter tensors: 13 , with 4 , 992 parameters

...

[ 2025-07-23 17:07:08,699647 ][ C ][ wordplay / trainer : 318 ] "devid='cuda:0'"

[ 2025-07-23 17:07:08,703940 ][ I ][ wordplay / trainer : 358 ] • self. model = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 1024 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-23 17:07:08,731597 ][ I ][ wordplay / trainer : 359 ] • self. grad_scaler = < torch.cuda.amp.grad_scaler.GradScaler object at 0x7cbd3c9a85d0 >

[ 2025-07-23 17:07:08,737375 ][ I ][ wordplay / trainer : 360 ] • self. model_engine = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 1024 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-23 17:07:08,760469 ][ I ][ wordplay / trainer : 361 ] • self. optimizer = AdamW ( 
Parameter Group 0 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: True 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.1 

Parameter Group 1 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: True 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.0 
 )

## Prompt (**prior** to training)

In [6]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 17:07:10,765047 ][ I ][ tmp / ipython-input-6-3496000222 : 9 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 17:07:10,767795 ][ I ][ tmp / ipython-input-6-3496000222 : 10 : ezpz.log ] [ 'response' ] :

What is an LLM?ouuu'fU?UUUU-LLlVmoYY;?U$IMwwYDjMYYXSSdIss;I''DPOjHhooooMZtmkoGXjZ
BDDddZkydVPcM'MAWILMDDP' '' !A'Vzl;R
dtA$ttoXttJJffobJJ;b-vkwwJJOHHwQFccddlobAGGnM' '' $kW;kzZlSwZkAoR;wmooo$J-fffoYDd'UBooXYB;JSf?P'MJ..t'hPffID;R.XXo' '' SPZkXXXe'VS.JoMdkXSffo' 'RHQklK' 'UUUSoMn

## Train Model

|  name  |         description          |
|:------:|:----------------------------:|
| `step` |    Current training step     |
| `loss` |          Loss value          |
|  `dt`  |  Time per step (in **ms**)   |
| `sps`  |      Samples per second      |
| `mtps` |   (million) Tokens per sec   |
| `mfu`  | Model Flops utilization\[1\] |

^legend: #tbl-legend

\[1\] in units of A100 `bfloat16` peak FLOPS

In [7]:
trainer.config.device_type

'cuda'

In [8]:
from rich import print

print(trainer.model)

GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 1024 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

## (partial) Training:

We’ll first train for 500 iterations and then evaluate the models
performance on the same prompt:

> What is an LLM?

In [9]:
trainer.train(train_iters=500)

Training Legend 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ abbr ┃ desc ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ step │ Current training iteration │
│ loss │ Loss value │
│ dt │ Elapsed time per training step │
│ dtf │ Elapsed time per forward step │
│ dtb │ Elapsed time per backward step │
│ sps │ Samples per second │
│ sps_per_gpu │ Samples per second (per GPU) │
│ tps │ Tokens per second │
│ tps_per_gpu │ Tokens per second (per GPU) │
│ mfu │ Model flops utilization │
└─────────────┴────────────────────────────────┘

[ 2025-07-23 17:07:12,567707 ][ I ][ wordplay / trainer : 827 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 17:07:12,572514 ][ I ][ wordplay / trainer : 831 ] [ 'response' ] :

What is an LLM?ZIoZo-om';- 'MAhB,RcOVP!JJhhkkJnnUzI''&D&jH!ddWJJhfUUVkRhZoZ:MoJRtDjkkhhdMM' Sdd-dbUoXXLSfyXXXRb3ZOS' '$!o&&jnVJ3MMkjJ' Mffe-cm..J3Oa;'$hooJ3z!jUSDn
 'DqBJtHH;!ozZIZokzoooYlMKLJm.DDmkkXRX' NnhMSccJsH;Ude.tRzDoUtm'JmCd;Jd&j'Qo& '$$DAJTPPVv&j' jjtmmtdls;wNNoooJ3$DDJ

[ 2025-07-23 17:08:14,213943 ][ I ][ wordplay / trainer : 894 ] step = 10 loss = 3.28901 dt = 0.388647 dtf = 0.0077605 dtb = 0.0102481 sps = 2.57303 sps_per_gpu = 2.57303 tps = 21078.3 tps_per_gpu = 21078.3 mfu = 0.622837

[ 2025-07-23 17:08:18,050755 ][ I ][ wordplay / trainer : 894 ] step = 20 loss = 2.82665 dt = 0.392386 dtf = 0.0123749 dtb = 0.0163346 sps = 2.54851 sps_per_gpu = 2.54851 tps = 20877.4 tps_per_gpu = 20877.4 mfu = 0.622244

...

[ 2025-07-23 17:15:02,033033 ][ I ][ wordplay / trainer : 894 ] step = 460 loss = 2.43378 dt = 0.379863 dtf = 0.0110734 dtb = 0.0147297 sps = 2.63253 sps_per_gpu = 2.63253 tps = 21565.6 tps_per_gpu = 21565.6 mfu = 0.631684

[ 2025-07-23 17:15:05,868916 ][ I ][ wordplay / trainer : 894 ] step = 470 loss = 2.41934 dt = 0.378727 dtf = 0.00844342 dtb = 0.0111405 sps = 2.64043 sps_per_gpu = 2.64043 tps = 21630.4 tps_per_gpu = 21630.4 mfu = 0.632431

[ 2025-07-23 17:15:09,703796 ][ I ][ wordplay / trainer : 894 ] step = 480 loss = 2.45929 dt = 0.382927 dtf = 0.00844033 dtb = 0.0114589 sps = 2.61146 sps_per_gpu = 2.61146 tps = 21393.1 tps_per_gpu = 21393.1 mfu = 0.632402

[ 2025-07-23 17:15:13,538234 ][ I ][ wordplay / trainer : 894 ] step = 490 loss = 2.4835 dt = 0.383195 dtf = 0.0079397 dtb = 0.0104966 sps = 2.60964 sps_per_gpu = 2.60964 tps = 21378.1 tps_per_gpu = 21378.1 mfu = 0.632332

[ 2025-07-23 17:15:17,374316 ][ I ][ wordplay / trainer : 894 ] step = 500 loss = 2.43789 dt = 0.382541 dtf = 0.00727845 dtb = 0.0100782 sps = 2.6141 sps_per_gpu = 2.6141 tps = 21414.7 tps_per_gpu = 21414.7 mfu = 0.632376

In [10]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 17:15:18,240721 ][ I ][ tmp / ipython-input-10-1425179755 : 12 : ezpz.log ] took: 0. 8133s

[ 2025-07-23 17:15:18,242822 ][ I ][ tmp / ipython-input-10-1425179755 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 17:15:18,245933 ][ I ][ tmp / ipython-input-10-1425179755 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM? burthilio s in o th twiser mbalilis ar sis alincore tt t mes mpresofo m whe hary ht ourighothast omy pomithe d?




Bu le wie IUTore ll ishath tes d fr irme nco s f maksere,
IAn he ise wicouss s, areatath meangre the, my hare wis pay toth laut athe s,
Ano

## Resume Training…

In [11]:
trainer.train()

[ 2025-07-23 17:15:19,128023 ][ I ][ wordplay / trainer : 827 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 17:15:19,129812 ][ I ][ wordplay / trainer : 831 ] [ 'response' ] :

What is an LLM?

POOSTOLENETES:
INIONEO: oft ffan yo pe hous tor ce me s here serste buthe he ase he


NENIO:
Whe arallin hatithofoull the, fousencay yont paris.
PENTER:
An o, that s f lllle ishan be be acer se war tha pe iopre is ore nckat, me my?

WI tofifre he llly po

[ 2025-07-23 17:16:16,858986 ][ I ][ wordplay / trainer : 762 ] Saving checkpoint to: / content

[ 2025-07-23 17:16:16,861272 ][ I ][ wordplay / trainer : 763 ] Saving model to: /content/ model.pth

[ 2025-07-23 17:16:17,190207 ][ I ][ wordplay / configs : 141 ] Appending / content to /content/wordplay/src/ckpts/ checkpoints.log

...

[ 2025-07-23 17:31:16,189335 ][ I ][ wordplay / trainer : 894 ] step = 1460 loss = 1.68073 dt = 0.381957 dtf = 0.00771424 dtb = 0.00991214 sps = 2.6181 sps_per_gpu = 2.6181 tps = 21447.4 tps_per_gpu = 21447.4 mfu = 0.63232

[ 2025-07-23 17:31:20,023731 ][ I ][ wordplay / trainer : 894 ] step = 1470 loss = 1.71749 dt = 0.389038 dtf = 0.0123934 dtb = 0.016246 sps = 2.57045 sps_per_gpu = 2.57045 tps = 21057.1 tps_per_gpu = 21057.1 mfu = 0.631309

[ 2025-07-23 17:31:23,858642 ][ I ][ wordplay / trainer : 894 ] step = 1480 loss = 1.72494 dt = 0.380766 dtf = 0.00802833 dtb = 0.0109163 sps = 2.62629 sps_per_gpu = 2.62629 tps = 21514.5 tps_per_gpu = 21514.5 mfu = 0.631751

[ 2025-07-23 17:31:27,693442 ][ I ][ wordplay / trainer : 894 ] step = 1490 loss = 1.72521 dt = 0.384513 dtf = 0.00979085 dtb = 0.0104102 sps = 2.60069 sps_per_gpu = 2.60069 tps = 21304.9 tps_per_gpu = 21304.9 mfu = 0.631529

[ 2025-07-23 17:31:31,528345 ][ I ][ wordplay / trainer : 894 ] step = 1500 loss = 1.70409 dt = 0.385203 dtf = 0.0109562 dtb = 0.0163935 sps = 2.59604 sps_per_gpu = 2.59604 tps = 21266.7 tps_per_gpu = 21266.7 mfu = 0.631217

## Evaluate Model

In [12]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-23 17:31:32,597792 ][ I ][ tmp / ipython-input-12-582817405 : 12 : ezpz.log ] took: 0. 9968s

[ 2025-07-23 17:31:32,599918 ][ I ][ tmp / ipython-input-12-582817405 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-23 17:31:32,601844 ][ I ][ tmp / ipython-input-12-582817405 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM? What, that the wild my lord,
And the shal to may so shal that the shall thee.

RICHARD:
What that there thee shal the const the shall so thine.

RICHARD:
The wil thee the shal shal that that the should.

RICHARD:
Then the shal too the show shal to thee.